In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pointbiserialr
#import researchpy as rp
%matplotlib inline

In [ ]:
pd.set_option("display.max_columns", None)
#df = pd.read_csv('./datasets/train_radiomics_occipital_CONTROL.csv')
df = pd.read_csv("datasets/train_radiomics_hipocamp.csv")
df2 = pd.read_csv("datasets/test_radiomics_hipocamp.csv")

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['Transition'].value_counts()

In [ ]:
df.shape

Tipos das colunas |
----------

In [ ]:
tipos = df.dtypes
contagem_por_tipo = tipos.value_counts()
for tipo, contagem in contagem_por_tipo.items():
    print(f"{tipo}: {contagem} colunas")

In [ ]:
tipos = df.dtypes
colunas_por_tipo = {}
for tipo in tipos.unique():
    colunas_por_tipo[tipo] = list(tipos[tipos == tipo].index)

for tipo, colunas in colunas_por_tipo.items():
    print(f"\n{tipo}:\n")
    for coluna in colunas:
        print(f"-> {coluna}")

Verificar valores nulos |
----------

In [ ]:
if df.isna().any().any():
    null_columns = df.columns[df.isna().any()] 
    for col in null_columns:
        print(f"Coluna '{col}' tem {df[col].isna().sum()} valores nulos")
else:
    print("No nulls")

In [ ]:
if df.isna().any(axis=1).any():
    null_rows = df.index[df.isna().any(axis=1)]
    for row in null_rows:
        print(f"Linha {row} tem {df.iloc[row].isna().sum()} valores nulos.")
else:
    print("Nenhuma linha contém valores nulos.")


Estatísticas por coluna |
----------

In [ ]:
df.describe()

Transformar o atributo Transition em 0/1/2/3/4

In [ ]:
target_mapping = {'AD-AD': 0, 'CN-CN': 1, 'MCI-AD': 2, 'MCI-MCI': 3, 'CN-MCI': 4}
if 'Transition' in df.columns:
    df['Transition'] = df['Transition'].map(target_mapping)

Valores mais baixos

In [ ]:
correlations = df.corr(numeric_only=True)['Transition']
#Ordenar
correlation_sorted = correlations.abs().sort_values().index
#print personalizado
correlation_list = [f"{col} ----> {correlations[col]}" for col in correlation_sorted]
print("\n".join(correlation_list[:20]))


Valores mais altos

In [ ]:
correlations = df.corr(numeric_only=True)['Transition']
#Ordenar
correlation_sorted = correlations.abs().sort_values(ascending=False).index
#print personalizado
correlation_list = [f"{col} ----> {correlations[col]}" for col in correlation_sorted]
print("\n".join(correlation_list[:20]))

#### Missing Values

In [ ]:
missing_columns = df.columns[df.isna().any()]

print("Colunas com missing values:", missing_columns.tolist())

#### Valores duplicados

In [ ]:
if (df.duplicated().sum() == 0):
    print ("Há um total de 0 linhas duplicadas")
else:
    print ("Há um total de" + df.duplicated().sum() + "linhas duplicadas")

In [ ]:
# Contar quantas colunas estão duplicadas
duplicadas = df.T.duplicated().sum()
print(f"Há um total de {duplicadas} colunas duplicadas.")


Obter os outliers |
------

In [ ]:
# pegamos so nas colunas numericas
colunas_numericas = df.select_dtypes(include=['float64', 'int64']).columns
# identificamos outliers em cada coluna
outlier_info = {}
for coluna in colunas_numericas:
    Q1 = df[coluna].quantile(0.25)
    Q3 = df[coluna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    # Número de valores fora dos limites inferior e superior
    outliers = df[(df[coluna] < limite_inferior) | (df[coluna] > limite_superior)][coluna]
    #outlier_info: Um dicionário onde a chave é o nome da coluna, e o valor é o número de outliers nela.
    outlier_info[coluna] = len(outliers)


# Converter números absolutos em percentuais
total_linhas = len(df)
# Um novo dicionário que armazena o percentual de outliers para cada coluna.
outlier_percentual = {col: count / total_linhas for col, count in outlier_info.items()}

In [ ]:
print("Outliers absolutos:", outlier_info)
print("Outliers percentuais:", outlier_percentual)

# Gráficos

In [ ]:
transition_counts = df['Transition'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=transition_counts.index, y=transition_counts.values, palette="Set1")
plt.xlabel('Transition')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.title('Distribuição de Transition')
plt.show()


In [ ]:
sex = df['Sex'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=sex.index, y=sex.values, palette="Set1")
plt.xlabel('Sexo')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.title('Sexo')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.histplot(df['Age'], bins=30, kde=False, color="blue", edgecolor="black")
plt.xlabel('Age')
plt.ylabel('Count')
plt.title('Casos p/idade')


plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x=df['Transition'],y=df['Age'],palette='deep')
plt.xlabel('Transition',fontsize=15)
plt.ylabel('Idade',fontsize=15)
plt.title('Distribuição de Transition por Idade')
plt.show()


In [ ]:

transition_counts = df.groupby(['Transition', 'Sex']).size().reset_index(name='Count')

plt.figure(figsize=(15, 5))
sns.barplot(x='Transition', y='Count', hue='Sex', data=transition_counts, palette="Set1")
plt.xlabel('Transition')
plt.ylabel('Count')
plt.xticks(rotation=45)
plt.title('Distribuição de Transition por Sexo')
plt.show()

In [ ]:
intervalos = [(0, 1), (1,2), (2,3), (3,4), (4,5), (5,100)]

contagem_intervalos = {f"{start}-{end}%": 0 for start, end in intervalos}

for percent in outlier_percentual.values():
    percentual = round(percent * 100)  
    for start, end in intervalos:
        if start <= percentual < end:
            contagem_intervalos[f"{start}-{end}%"] += 1
            break

labels = list(contagem_intervalos.keys()) 
sizes = list(contagem_intervalos.values()) 

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=plt.cm.Paired.colors)
plt.title("Percentagem de colunas com X intervalo de percentagem de outliers")
plt.show()


In [ ]:
numeric_df = df.select_dtypes(include=['number'])
correlation_matrix = numeric_df.corr()
target_corr = correlation_matrix['Transition'].abs()
top_10_attributes = target_corr.drop(labels=['Transition']).nlargest(10)
print("Os 10 atributos com maior correlação absoluta em relação ao atributo-alvo:")
print(top_10_attributes)

In [ ]:
selected_attributes = top_10_attributes.index.tolist()
heatmap_matrix = correlation_matrix.loc[selected_attributes + ['Transition'], selected_attributes + ['Transition']]
plt.figure(figsize=(10, 8))
sns.heatmap(heatmap_matrix, cmap='coolwarm', annot=True, fmt=".2f", cbar=True)
plt.title('Heatmap das 10 Principais Correlações com o Transition')
plt.show()
